# Activity Log - Export

**Author**: Melissa Coates  

**Last updated and tested**: September 29, 2022 

**Purpose:** This notebook exports data from the [Power BI activity log](https://docs.microsoft.com/en-us/power-bi/admin/service-admin-auditing).

**Type of authentication**: You can log in with either:

- A domain user account (requires Power BI administrator permissions), or
- A service principal (with permission to run admin APIs)

**How data is being accessed:** The notebook uses:

- Power BI Management Module for authentication: Connect-PowerBIServiceAccount
- Power BI Management Module for extracting the activity log data: Get-PowerBIActivityEvent

**Type of script**: Interactive, ad hoc (non-scheduled). This notebook is a guided learning experience to get familiar with how to query the activity log. It's not a production-ready script. This script accompanies the Auditing & Monitoring content found in the [Power BI Deployment and Governance](https://www.coatesds.com/power-bi-deployment-and-governance) course. 

More information, including prerequisites, is included at the bottom of this notebook.

## **Log in to the Power BI Service**

* * *

**Input before executing:**

- **Type of authentication:** 
    - **Line 2 - Choice for type of auth:** Input either '**User**' or '**User\_MFA**' or '**SP**' without the quotes. This tells the script whether or not to log in as a user or a service principal.
    
    - - **User**: If multi-factor authentication is not enabled for the user account. The notebook will prompt you to type in your password.
        - **User\_MFA**: If multi-factor authentication is enabled for the user account. You'll be directed to a web browser to complete the login.
        - **SP**: If you're using a service principal.
- **If using user authentication:**
    - **Line 3 - Email address:** Input the email address for the user. This user must be a Power BI administrator.
- **If using service principal authentication:**
    - **Line 4 - Azure AD app ID**: Input the app ID (aka client ID). 
    - **Line 5 - Azure tenant ID**: Input the tenant ID (aka directory ID).

**Interactive prompts:**

The script will interactively prompt you for the sensitive information (because hard-coding of passwords and secrets isn't secure):

- **If using user authentication:**
    - It will ask for the user's password.
- **If using service principal authentication:**
    - It will ask for the app secret.

**Comments:**

- This script uses the PowerShell cmdlet: [<u>Connect-PowerBIServiceAccount</u>](https://docs.microsoft.com/en-us/powershell/module/microsoftpowerbimgmt.profile/connect-powerbiserviceaccount).
- After signing in, the access token is active for one hour.
- If you belong to more than one tenant, add the Tenant parameter (to Connect-PowerBIServiceAccount) to specify a tenant ID when you log in.

In [ ]:
#-----------------INPUT AREA--------------------------------------------
[string]$TypeOfAuth = 'User'  #Options: User --or-- User_MFA --or-- SP
[string]$DomainUserEmailAddr = 'user@domain.com'
[string]$AzureADAppID = ''
[string]$AzureTenantID = ''
#-----------------------------------------------------------------------

if ($TypeOfAuth -eq 'User')
{
    Write-Verbose "Domain user authentication will be used." -Verbose
    [securestring]$DomainUserPW = Read-Host -Prompt "Input password for $DomainUserEmailAddr" -AsSecureString
    [pscredential]$CredentialObj = New-Object System.Management.Automation.PSCredential($DomainUserEmailAddr, $DomainUserPW)  
    Connect-PowerBIServiceAccount -Credential $CredentialObj
}
elseif ($TypeOfAuth -eq 'User_MFA')
{
    Write-Verbose "Domain user with multi-factor authentication will be used." -Verbose
    Connect-PowerBIServiceAccount
}
elseif ($TypeOfAuth -eq 'SP')
{
    Write-Verbose "Service principal authentication will be used." -Verbose
    [securestring]$AzureADAppSecret = Read-Host -Prompt "Input secret for $AzureADAppID" -AsSecureString
    [pscredential]$CredentialObj = New-Object System.Management.Automation.PSCredential($AzureADAppID, $AzureADAppSecret)  
    Connect-PowerBIServiceAccount -Credential $CredentialObj -ServicePrincipal -Tenant $AzureTenantID
}
else 
{
    Write-Verbose "There's a problem. The Type of Auth variable input was $TypeOfAuth. Expected values are User or SP." -Verbose
}

## **Export Activity Log: All Activities, All Users, Up To 30 Days Back**

* * *

## <span style="font-size: 14px;">This script will allow you to extract the raw data for all activity log data, for all users, up to a maximum of 30 days back. This is useful when you want to export the data so it can be viewed/used in other tools.</span>

**Input before executing:**

- **Line 2: Number of days to extract:** Data up to 30 days in the past may be retrieved.
- **Line 3: Location where to export the files:** This folder must already exist.

**Comments**:

- **Cmdlet used:** This script uses the PowerShell cmdlet: [<u>Get-PowerBIActivityEvent</u>](https://docs.microsoft.com/en-us/powershell/module/microsoftpowerbimgmt.admin/get-powerbiactivityevent). 
- **One file per day:** It exports one JSON file per day (based on UTC time). Saving files, partitioned by day, is very useful when you want to store raw data over time. 
- **File name:** The format of the file name is: PBIActivityEvents-\<DateOfEvents\>-\<DateTimeDataWasExtracted\>.json - the suffix includes a datetime of when the extract was run is included so that if there are multiple executions of the same day, you know which one is newer (occasionally there are late-arriving events but that's not typically a big issue).
- **File location**: For simplicity in this demo/training script, a local file path is used. When using this "for real" you want to retain the raw data permanently in a location that's secure and backed up. If you need to rely on the log files for formal security audits, store the files in an immutable location (so the files can't be altered or deleted),.
- **Loop through range of time:** The range of time is specified with the first variable. It counts backwards from yesterday to determine how many days it should loop through. A maximum of one day is allowed to be extracted at once from the cmdlet (and its underlying API). That's why the loop runs once per day. 
- **Range of days:** The range will conclude with yesterday (UTC time). Today is intentionally ignored -- goal is to avoid having a partial day's worth of data in a file. (You might pull today's records in an ad hoc script, but not when exporting the data.)
- **JSON file format:** The JSON format is used because, over time, new data elements will be introduced by Power BI. A flexible schema is necessary to accomodate that so that you don't miss when new types of data get added (i.e., you want to parse the JSON and retrieve what you're interested in \*after\* the raw data has been extracted, not during the data extraction). It is a best practice to store the raw data as-is, in an immutable location, so the data is reliable for auditing.
- **Messages**: The verbose messages are displayed so you know where the script is at.

In [ ]:
#-----------------INPUT AREA-----------------
[int32]$NbrDaysDaysToExtract = 15  #Maximum is 30 days back
[string]$ExportFileLocation = 'C:\Power-BI-Raw-Data\Activity-Log'
#--------------------------------------------

[datetime]$YesterdayUTC = (([datetime]::Today.ToUniversalTime()).Date).AddDays(-1) #Begin with yesterday, rather than today, to ensure full day results are obtained
[string]$DateTimeFileWrittenUTCLabel = ([datetime]::Now.ToUniversalTime()).ToString("yyyyMMddHHmm")

#Loop through each of the days to be extracted (<Initilize> ; <Condition> ; <Repeat>)
for($Loop=0 ; $Loop -lt $NbrDaysDaysToExtract ; $Loop++)
{
    [datetime]$DateToExtractUTC=$YesterdayUTC.AddDays(-$Loop).ToString("yyyy-MM-dd")

    [string]$DateToExtractLabel=$DateToExtractUTC.ToString("yyyy-MM-dd")
    
    [string]$ExportFileName = 'PBIActivityEvents-' + ($DateToExtractLabel -replace '-', '') + '-' + $DateTimeFileWrittenUTCLabel + '.json' 

    [psobject]$Events=Get-PowerBIActivityEvent -StartDateTime ($DateToExtractLabel+'T00:00:00.000') -EndDateTime ($DateToExtractLabel+'T23:59:59.999')
  
    $Events | Out-File "$ExportFileLocation\$ExportFileName"

    Write-Verbose "File written: $ExportFileName" -Verbose 
}
Write-Verbose "Extract of Power BI activity events is complete." -Verbose 

## **Disconnect from the Power BI Service**

* * *

It's useful to add this at the end of automation scripts, as a matter of good housekeeping. If you don't log out (or refresh your access token), you'll be automatically disconnected after 1 hour.

In [ ]:
Disconnect-PowerBIServiceAccount

## **More Information About This File**

* * *

## <span style="font-size:14px;">Prerequisites for running this script:</span>

- The [Power BI Management module](https://www.powershellgallery.com/packages/MicrosoftPowerBIMgmt) for PowerShell needs to be installed.
    
- The [.NET Interactive Notebooks](https://marketplace.visualstudio.com/items?itemName=ms-dotnettools.dotnet-interactive-vscode) extension needs to be installed and enabled in VS Code and/or Azure Data Studio.
    
- Either a user account with Power BI administrator permissions, or a service principal with permission to run admin APIs.
    

This script **interactively prompts** for the sensitive information to be passed into a credential object. Therefore, it's only useful for ad hoc purposes. A different technique is used for securely storing the credentials when a script is to be operationalized and scheduled.

The original / most recent version of this notebook can be found on Melissa Coates' [Power-BI-Admin repository in GitHub](https://github.com/sqlchick/Power-BI-Admin).